## <a id='Задание_1_14'>Задание № 1</a>

Обучить модель GPT для генерации своих цитат.

Взять данные из https://www.kaggle.com/datasets/mrapplexz/bashim-quotes

### <a id='Данные_1_14'>Загрузка и подготовка данных</a>

In [2]:
#!g2.2
DATASET_PATH = '/home/jupyter/work/resources/14_Transfer_learning/Data/dataset.jsonl'
train_path = '/home/jupyter/work/resources/14_Transfer_learning/Data/train_dataset.txt'
test_path = '/home/jupyter/work/resources/14_Transfer_learning/Data/test_dataset.txt'
RES_PATH = '/home/jupyter/work/resources/14_Transfer_learning/results/' # Папка результатов

# Открываем файл
with open(DATASET_PATH) as f: 
   # Считываем .jsonl file как Pandas DataFrame
    df = pd.read_json(DATASET_PATH, lines=True).set_index('id')
    
df.sample(1)

,date,rating,text
id,,,
448538,2018-01-11 07:45:00+00:00,4664.0,"Смотрим с друзьями ""Трое из Простоквашино"". Сц..."


In [3]:
#!g2.2
# Количество данных
df.shape

(81497, 3)

In [4]:
#!g2.2
# Берем только часть данных столбца text (сокращаем объем данных)
data = df.loc[:3500, 'text']

In [5]:
#!g2.2
# Разделяем данные на train и test
train, test = train_test_split(data, test_size=0.15)

In [6]:
#!g2.2
# Весь text из каждого набора данных препроцессим и собираем в отдельные файлы
def build_text_files(data_json, dest_path):
    f = open(dest_path, 'w')
    data = ''
    for texts in data_json:
        summary = str(texts).strip() # удаление символов с начала и с конца строки
        summary = re.sub(r"\[\s*\w+\s*\]", "", summary) # убираем слова в квадратных скобках
        summary = re.sub(r"<\s*\w+\s*>", "", summary) # убираем слова в скобках типа <>
        summary = re.sub(r"\s", " ", summary) # заменяем любые пробельные символы одиночным пробелом
        summary = re.sub(r"\*+", "", summary) # убираем звездочки *
        data += summary + "  "
    f.write(data)

build_text_files(train, train_path)
build_text_files(test, test_path)

In [7]:
#!g2.2
with open(train_path) as f:
    train_dataset = f.read()
# train_dataset

In [8]:
#!g2.2
print("Train dataset length: "+ str(len(train)))
print("Test dataset length: "+ str(len(test)))

Train dataset length: 983
Test dataset length: 174


In [9]:
#!g2.2
# Загрузка токенайзера
model_name = "sberbank-ai/rugpt3small_based_on_gpt2"
tokenizer = AutoTokenizer.from_pretrained(model_name)

Downloading:   0%|          | 0.00/608 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.63M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.21M [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [10]:
#!g2.2
# Функция загрузки датасетов
def load_dataset(train_path, test_path, tokenizer):
    train_dataset = TextDataset(
          tokenizer=tokenizer,
          file_path=train_path,
          block_size=128)

    test_dataset = TextDataset(
          tokenizer=tokenizer,
          file_path=test_path,
          block_size=128)

    data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)
    
    return train_dataset, test_dataset, data_collator

In [11]:
#!g2.2
# Создаем тренировочный и тестовый датасеты
train_dataset, test_dataset, data_collator = load_dataset(train_path, test_path, tokenizer)

### <a id='Модель_1_14'>Модель</a>

In [12]:
#!g2.2
# Загрузка и обучение модели
model = AutoModelForCausalLM.from_pretrained(model_name)

Downloading:   0%|          | 0.00/526M [00:00<?, ?B/s]

In [13]:
#!g2.2
# Аргументы для обучения
training_args = TrainingArguments(
    output_dir=RES_PATH, #The output directory
    overwrite_output_dir=True, #overwrite the content of the output directory
    num_train_epochs=3, # number of training epochs
    per_device_train_batch_size=4, # batch size for training
    per_device_eval_batch_size=4,  # batch size for evaluation
    eval_steps = 400, # Number of update steps between two evaluations.
    save_steps=800, # after # steps model is saved
    warmup_steps=500,# number of warmup steps for learning rate scheduler
    )

In [14]:
#!g2.2
# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    eval_dataset=test_dataset
)

In [15]:
#!g2.2
# Обучение модели
trainer.train()

Step,Training Loss


Step,Training Loss


Step,Training Loss


Step,Training Loss


Step,Training Loss


...

Step,Training Loss


Step,Training Loss


***** Running training *****
  Num examples = 462
  Num Epochs = 3
  Instantaneous batch size per device = 4
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 174


Training completed. Do not forget to share your model on huggingface.co/models =)




Step,Training Loss


TrainOutput(global_step=174, training_loss=4.57403248754041, metrics={'train_runtime': 13.6639, 'train_samples_per_second': 101.435, 'train_steps_per_second': 12.734, 'total_flos': 90537689088000.0, 'train_loss': 4.57403248754041, 'epoch': 3.0})

In [16]:
#!g2.2
# Сохранение модели для последующей перезагрузки (from_pretrained())
trainer.save_model()

Saving model checkpoint to /home/jupyter/work/resources/14_Transfer_learning/results/
Configuration saved in /home/jupyter/work/resources/14_Transfer_learning/results/config.json
Model weights saved in /home/jupyter/work/resources/14_Transfer_learning/results/pytorch_model.bin


In [17]:
#!g2.2
# Сохранение словаря токенайзера с добавленными токенами, специальными токенами атрибутами класса и входными установками токенайзера
tokenizer.save_pretrained(Path(RES_PATH).joinpath('gpt2-chief'))

# Сохранение модели и ее конфигурационного файла
model.save_pretrained(Path(RES_PATH).joinpath('model_gpt_chf'))

tokenizer config file saved in /home/jupyter/work/resources/14_Transfer_learning/results/gpt2-chief/tokenizer_config.json
Special tokens file saved in /home/jupyter/work/resources/14_Transfer_learning/results/gpt2-chief/special_tokens_map.json
Configuration saved in /home/jupyter/work/resources/14_Transfer_learning/results/model_gpt_chf/config.json
Model weights saved in /home/jupyter/work/resources/14_Transfer_learning/results/model_gpt_chf/pytorch_model.bin


### <a id='Генерация_1_14'>Генерация текста</a>

In [18]:
#!g2.2
# Перезагрузка токенизатора
tokenizer = AutoTokenizer.from_pretrained(Path(RES_PATH).joinpath('gpt2-chief'))

# Загрузка модели
model1 = AutoModelForCausalLM.from_pretrained(Path(RES_PATH).joinpath('model_gpt_chf'))

loading file /home/jupyter/work/resources/14_Transfer_learning/results/gpt2-chief/vocab.json
loading file /home/jupyter/work/resources/14_Transfer_learning/results/gpt2-chief/merges.txt
loading file /home/jupyter/work/resources/14_Transfer_learning/results/gpt2-chief/tokenizer.json
loading file /home/jupyter/work/resources/14_Transfer_learning/results/gpt2-chief/added_tokens.json
loading file /home/jupyter/work/resources/14_Transfer_learning/results/gpt2-chief/special_tokens_map.json
loading file /home/jupyter/work/resources/14_Transfer_learning/results/gpt2-chief/tokenizer_config.json
loading configuration file /home/jupyter/work/resources/14_Transfer_learning/results/model_gpt_chf/config.json
Model config GPT2Config {
  "_name_or_path": "/home/jupyter/work/resources/14_Transfer_learning/results/model_gpt_chf",
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 50256,
  "embd_pdrop": 0.1,
  "eos_token_id": 50256,

In [19]:
#!g2.2
# Генерация текста
prefix = "привет "
tokens = tokenizer(prefix, return_tensors='pt')
size = tokens['input_ids'].shape[1]
output = model1.generate( 
    **tokens, 
    #end_token=end_token_id,
    do_sample=False,
    max_length=size+50, 
    repetition_penalty=5., 
    temperature=0.5,
    num_beams=10,
    pad_token_id=tokenizer.eos_token_id
)

decoded = tokenizer.decode(output[0])
result = decoded[len(prefix):]
print(prefix + result)

привет ^_^   я не знаю, что это такое и как оно работает. но у меня есть подозрение, что эта штука может работать только в том случае, если она подключена к интернету с помощью USB-кабеля (впрочем,


# <a id='Урок_14'>Домашнее задание № 14. Transfer learning</a>

## <a id='Подготовка_14'>Подготовка</a>

In [1]:
#!g2.2
# Импорт библиотек и модулей
import pandas as pd
from sklearn.model_selection import train_test_split
import re
from transformers import AutoTokenizer, TextDataset, DataCollatorForLanguageModeling
from transformers import T5ForConditionalGeneration, Trainer, TrainingArguments, AutoModelForCausalLM
from corus import load_lenta2
from datasets import Dataset, load_from_disk
import torch
from pathlib import Path
import os
import warnings
warnings.filterwarnings("ignore")

## <a id='Задание_2_14'>Задание № 2</a>

Обучить модель T5/ или GPT для генерации заголовков для статей.
Взять новостные данные из https://github.com/natasha/corus load_lenta2, нам понадобиться сам текст и заголовок.

### <a id='Данные_2_14'>Загрузка и подготовка данных</a>

In [2]:
#!g2.2
# Скачивание данных
!wget https://github.com/yutkin/Lenta.Ru-News-Dataset/releases/download/v1.1/lenta-ru-news.csv.bz2

 33s
 12450K .......... .......... .......... .......... ..........  3%  494M 33s
 12500K .......... .......... .......... .......... ..........  3%  451M 33s
 12550K .......... .......... .......... .......... ..........  3%  501M 32s
 12600K .......... .......... .......... .......... ..........  3%  441M 32s
 12650K .......... .......... .......... .......... ..........  3%  631M 32s
 12700K .......... .......... .......... .......... ..........  3%  646M 32s
 12750K .......... .......... .......... .......... ..........  3%  619M 32s
 12800K .......... .......... .......... .......... ..........  3%  441M 32s
 12850K .......... .......... .......... .......... ..........  3%  444M 32s
 12900K .......... .......... .......... .......... ..........  3%  496M 32s
 12950K .......... .......... .......... .......... ..........  3%  660M 31s
 13000K .......... .......... .......... .......... ..........  3% 18.3M 31s
 13050K .......... .......... .......... .......... ..........  3%  278

In [3]:
#!g2.2
# Загрузка данных
path = 'lenta-ru-news.csv.bz2'
records = load_lenta2(path)
next(records)

LentaRecord(
    url='https://lenta.ru/news/1914/09/16/hungarnn/',
    title='1914. Русские войска вступили в\xa0пределы Венгрии  ',
    text='Бои у Сопоцкина и Друскеник закончились отступлением германцев. Неприятель, приблизившись с севера к Осовцу начал артиллерийскую борьбу с крепостью. В артиллерийском бою принимают участие тяжелые калибры. С раннего утра 14 сентября огонь достиг значительного напряжения. Попытка германской пехоты пробиться ближе к крепости отражена. В Галиции мы заняли Дембицу. Большая колонна, отступавшая по шоссе от Перемышля к Саноку, обстреливалась с высот нашей батареей и бежала, бросив парки, обоз и автомобили. Вылазки гарнизона Перемышля остаются безуспешными. При продолжающемся отступлении австрийцев обнаруживается полное перемешивание их частей, захватываются новые партии пленных, орудия и прочая материальная часть. На перевале Ужок мы разбили неприятельский отряд, взяли его артиллерию и много пленных и, продолжая преследовать, вступили в пределы Венгрии

In [4]:
#!g2.2
for i, record in enumerate(records):
  print(record.title)
  break

1914. Празднование столетия М.Ю. Лермонтова отложено 


In [5]:
#!g2.2
for i, record in enumerate(records):
  print(record.text)
  break

Штабс-капитан П. Н. Нестеров на днях, увидев в районе Желтиева, в Галиции, летящий над нашим расположением австрийский аэроплан, собиравшийся бросить бомбы, взлетел на воздух, атаковал неприятеля и протаранил неприятельский аппарат, предотвратив жертвы в наших войсках. Сам Нестеров при этом погиб смертью героя. По словам доставленных в Киев пленных австрийских офицеров, всей неприятельской армии хорошо известно имя Нестерова. Во время воздушных разведок русских авиаторов австрийцы всегда безошибочно определяли, каким аппаратом управлял Нестеров. Когда показывался аэроплан-птица, красиво и вольно паривший в воздухе, австрийцы указывали — Das ist Nesteroff! Австрийцы боялись покойного, и все их усилия были направлены к прекращению его деятельности. За задержание отважного летчика была объявлена большая премия. Нестеров погиб в 27 лет. После Нестерова остались жена и двое детей — девочка, 5-ти лет, и мальчик, 3-х лет. Иллюстрированный журнал «Искры» № 35, сентябрь 1914 года  Песнь о Несте

In [6]:
#!g2.2
# Собираем название и текст новостей и преобразуем в датафрейм
data = [(record.title, record.text) for record in records]

In [7]:
#!g2.2
data[0]

('1914. Бульдог-гонец под Льежем ',
 'Фотограф-корреспондент Daily Mirror рассказывает случай, который порадует всех друзей животных. Лейтенант бельгийской артиллерии, руководивший обороной одного из фортов Льежа, ни за что не хотел расстаться с своей собакой — бульдогом. Когда пруссаки пробрались между фортов в самый город, офицеру пришло в голову доверить бульдогу письмо, в котором он посылал успокоительную весть своим родителям. Благородный пес честно исполнил свою миссию. Десять часов спустя бульдог проник обратно в форт и принес ответ. С этого момента бульдог стал настоящим гонцом. Много раз пробирался он через линии германских войск, неся на себе спрятанными в ошейнике шифрованные депеши.Журнал «Нива» №37, сентябрь 1914 года')

In [11]:
#!g2.2
df_news = pd.DataFrame({'title': [record[0] for record in data], 'text': [record[1] for record in data]})
df_news.head()

,title,text
0,1914. Бульдог-гонец под Льежем,Фотограф-корреспондент Daily Mirror рассказыва...
1,1914. Под Люблином пойман швабский зверь,"Лица, приехавшие в Варшаву из Люблина, передаю..."
2,"Космонавты сомневаются в надежности ""Мира""",Как стало известно агентству Ассошиэйтед Пресс...
3,Взрыв в центре Москвы: пострадало 30 человек,В зале игровых автоматов в третьем ярусе подзе...
4,Япония кредитует Россию на полтора миллиарда д...,Япония приняла решение разморозить кредиты Рос...


In [12]:
#!g2.2
# Размер датасета
df_news.shape

(800972, 2)

In [13]:
#!g2.2
# Максимальная длина текста и названия статьи
max(len(i) for i in df_news['text'].values), max(len(i) for i in df_news['title'].values)

(55386, 132)

In [14]:
#!g2.2
# Ограничиваем размер данных
data_news = df_news[:500]

In [15]:
#!g2.2
# Разделяем данные на train и test
df_train, df_test = train_test_split(data_news, test_size=0.15)

In [16]:
#!g2.2
# Складываем объекты в Dataset (для huggingface)
df_train.reset_index(drop=True, inplace=True)
df_test.reset_index(drop=True, inplace=True)

dataset_train = Dataset.from_pandas(df_train)
dataset_test = Dataset.from_pandas(df_test)
dataset_train, dataset_test

(Dataset({
     features: ['title', 'text'],
     num_rows: 425
 }),
 Dataset({
     features: ['title', 'text'],
     num_rows: 75
 }))

In [17]:
#!g2.2
# Создаем объект класса токенизатор
model_name = "IlyaGusev/rut5_base_sum_gazeta"
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Ограничиваем максимальную длину текста и новости
max_len_txt = 400
max_len_tlt = 50

# Токенизируем вход
def tokenize(batch):
    tokenized_input = tokenizer(batch['text'], padding='max_length', truncation=True, max_length=max_len_txt)
    tokenized_label = tokenizer(batch['title'], padding='max_length', truncation=True, max_length=max_len_tlt)
    tokenized_input['labels'] = tokenized_label['input_ids']

    return tokenized_input

# Токенизируем данные из датасетов
dataset_train = dataset_train.map(tokenize, batched=True, batch_size=8)
dataset_test = dataset_test.map(tokenize, batched=True, batch_size=8)

# Столбцы ['input_ids', 'attention_mask', 'labels'] переводим в формат numpy
dataset_train.set_format('numpy', columns=['input_ids', 'attention_mask', 'labels'])
dataset_test.set_format('numpy', columns=['input_ids', 'attention_mask', 'labels'])

Downloading:   0%|          | 0.00/279 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/808k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.25M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

Map:   0%|          | 0/425 [00:00<?, ? examples/s]

Map:   0%|          | 0/75 [00:00<?, ? examples/s]

In [18]:
#!g2.2
# Подготовленные датасеты
dataset_train, dataset_test

(Dataset({
     features: ['title', 'text', 'input_ids', 'attention_mask', 'labels'],
     num_rows: 425
 }),
 Dataset({
     features: ['title', 'text', 'input_ids', 'attention_mask', 'labels'],
     num_rows: 75
 }))

In [19]:
#!g2.2
# Сохранение подготовленных датасетов
dataset_train.save_to_disk('gazeta/train')
dataset_test.save_to_disk('gazeta/test')

Saving the dataset (0/1 shards):   0%|          | 0/425 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/75 [00:00<?, ? examples/s]

### <a id='Модель_2_14'>Модель</a>

In [26]:
#!g2.2
# Создаем экземпляр модели
model_name = "IlyaGusev/rut5_base_sum_gazeta"
model = T5ForConditionalGeneration.from_pretrained(model_name).to('cuda')

loading configuration file https://huggingface.co/IlyaGusev/rut5_base_sum_gazeta/resolve/main/config.json from cache at /tmp/xdg_cache/huggingface/transformers/426a325da473aa010e527ee99032b35ce9354913e38282d34e50dd75856c82f7.87df939950b4282d4195b92cd0a6209ec6d3d69e74b13a77d87890cf3a5ded7b
Model config T5Config {
  "_name_or_path": "cointegrated/rut5-base",
  "architectures": [
    "T5ForConditionalGeneration"
  ],
  "bos_token_id": 2,
  "d_ff": 2048,
  "d_kv": 64,
  "d_model": 768,
  "decoder_start_token_id": 2,
  "dropout_rate": 0.1,
  "eos_token_id": 1,
  "feed_forward_proj": "gated-gelu",
  "initializer_factor": 1.0,
  "is_encoder_decoder": true,
  "layer_norm_epsilon": 1e-06,
  "max_length": 200,
  "model_type": "t5",
  "num_beams": 5,
  "num_decoder_layers": 12,
  "num_heads": 12,
  "num_layers": 12,
  "output_past": true,
  "pad_token_id": 0,
  "relative_attention_num_buckets": 32,
  "tie_word_embeddings": false,
  "tokenizer_class": "T5Tokenizer",
  "torch_dtype": "float32",
  "

In [27]:
#!g2.2
# Настройки обучения
output_dir = 'gazeta/output'

training_args = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=3,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    # eval_accumulation_steps=1, # Number of eval steps to keep in GPU (the higher, the mor vRAM used)
    # prediction_loss_only=True, # If I need co compute only loss and not other metrics, setting this to true will use less RAM
    # learning_rate=0.00001,
    # evaluation_strategy='steps', # Run evaluation every eval_steps
    save_steps=1000, # How often to save a checkpoint
    # save_total_limit=1, # Number of maximum checkpoints to save
    remove_unused_columns=True, # Removes useless columns from the dataset
    # run_name='run_gazeta', # Wandb run name
    # logging_steps=500, # How often to log loss to wandb
    eval_steps=500, # How often to run evaluation on the val_set
    # logging_first_step=False, # Whether to log also the very first training step to wandb
    # load_best_model_at_end=True, # Whether to load the best model found at each evaluation.
    # metric_for_best_model="loss", # Use loss to evaluate best model.
    # greater_is_better=False # Best model is the one with the lowest loss, not highest.
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [28]:
#!g2.2
# Загружаем датасеты
dataset_train = load_from_disk("gazeta/train")
dataset_test = load_from_disk("gazeta/test")
dataset_train, dataset_test

(Dataset({
     features: ['title', 'text', 'input_ids', 'attention_mask', 'labels'],
     num_rows: 425
 }),
 Dataset({
     features: ['title', 'text', 'input_ids', 'attention_mask', 'labels'],
     num_rows: 75
 }))

In [29]:
#!g2.2
# Обучение модели
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset_train,
    eval_dataset=dataset_test
)

trainer.train()

Step,Training Loss


Step,Training Loss


Step,Training Loss


Step,Training Loss


Step,Training Loss


...

Step,Training Loss


Step,Training Loss


The following columns in the training set  don't have a corresponding argument in `T5ForConditionalGeneration.forward` and have been ignored: text, title.
***** Running training *****
  Num examples = 425
  Num Epochs = 3
  Instantaneous batch size per device = 2
  Total train batch size (w. parallel, distributed & accumulation) = 4
  Gradient Accumulation steps = 1
  Total optimization steps = 321


Training completed. Do not forget to share your model on huggingface.co/models =)




Step,Training Loss


TrainOutput(global_step=321, training_loss=9.390902605383568, metrics={'train_runtime': 60.6037, 'train_samples_per_second': 21.038, 'train_steps_per_second': 5.297, 'total_flos': 677081548800000.0, 'train_loss': 9.390902605383568, 'epoch': 3.0})

In [30]:
#!g2.2
# Сохранение модели
trainer.save_model(output_dir + '/model')

Saving model checkpoint to gazeta/output/model
Configuration saved in gazeta/output/model/config.json
Model weights saved in gazeta/output/model/pytorch_model.bin


### <a id='Генерация_2_14'>Генерация заголовков статей</a>

In [31]:
#!g2.2
# Тестовые данные
INX = 10
input_text = dataset_test['text'][INX]
input_title = dataset_test['title'][INX]

with torch.no_grad():
    tokenized_text = tokenizer(input_text, truncation=True, padding=True, return_tensors='pt').to('cuda')

    source_ids = tokenized_text['input_ids'].to(dtype = torch.long)
    source_mask = tokenized_text['attention_mask'].to(dtype = torch.long)

    generated_ids = model.generate(
        input_ids = source_ids,
        attention_mask = source_mask, 
        max_length=512,
        num_beams=7,
        temperature = 1.3,
        repetition_penalty=1, 
        length_penalty=1, 
        early_stopping=True,
        no_repeat_ngram_size=2
    )

    pred = tokenizer.decode(generated_ids[0], skip_special_tokens=True, clean_up_tokenization_spaces=True)

print("Text:\n" + input_text)
print("Real title: " + input_title)
print("Pred title: " + pred)

Text:
Сегодня утром в районах дагестанских селений Чабанмахи и Карамахи возобновились активные боевые действия подразделений внутренних войск МВД.  Как сообщили "Интерфаксу" во временном пресс-центре объединенной группировки Минобороны РФ в Дагестане, они встречают сильное огневое сопротивление со стороны экстремистов. Всего, по имеющимся в пресс-центре сведениям, группировка исламских боевиков в этом районе превышает 500 человек. Вчера в результате ожесточенного боя ими была выведена из строя боевая машина пехоты федеральных сил. Погибли двое и ранены еще двое военнослужащих. Вертолеты федеральных сил вчера нанесли 8 ракетных ударов по позициям боевиков в районах горы Чабан, а также в километре восточнее села Дуранги и в полутора километрах от того села. По боевым позициям исламистов также вела "сосредоточенный огонь" артиллерия.Удалось перехватить радиосообщений исламистов, в которых они просят о помощи у своих единомышленников в Чечне и Дагестане. 30 августа при продвижении из район